<a href="https://colab.research.google.com/github/AquaFAN171029/Qwen-Fire/blob/main/Qwen_image_testing%203.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install diffusers>=0.35.1 transformers accelerate safetensors pillow

import os, torch
from PIL import Image, ImageDraw
from diffusers import QwenImageEditPipeline

from PIL import Image
import numpy as np
import cv2
import os, json

edit = QwenImageEditPipeline.from_pretrained(
    "Qwen/Qwen-Image-Edit",
    torch_dtype=torch.bfloat16
    )
edit.enable_model_cpu_offload()
edit.text_encoder.to("cpu")
print("loaded")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/512 [00:00<?, ?B/s]

Fetching 36 files:   0%|          | 0/36 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

loaded


In [ ]:

base = Image.open("back.png").convert("RGB")
x1,y1,x2,y2 = 846,552,967,971
pad = 12
x1p=max(0,x1-pad); y1p=max(0,y1-pad); x2p=min(base.width,x2+pad); y2p=min(base.height,y2+pad)
roi = base.crop((x1p,y1p,x2p,y2p))

#在 ROI上插入灭火器
prompt = ("Insert a red metal fire extinguisher storage box with transparent glass door, placed on the floor beside a wall, labeled in both Chinese and English")
out_roi = edit(
    image=roi,
    prompt=prompt,
    negative_prompt="cartoon, painting, logo, watermark, blurry, deformed",
    true_cfg_scale=4.0,
    num_inference_steps=50,
    generator=torch.manual_seed(42)
).images[0]

# 转坐标到int，判定坐标范围
W, H = base.size
x1  = int(x1);  y1  = int(y1);  x2  = int(x2);  y2  = int(y2)
x1p = max(0, int(x1p)); y1p = max(0, int(y1p))
x2p = min(W, int(x2p)); y2p = min(H, int(y2p))

#以 roi 的真实尺寸，重新推导右下角
rw, rh = roi.size
x2p = x1p + rw
y2p = y1p + rh
assert x2p <= W and y2p <= H, "ROI Error"

#开始贴回：

base_np = np.array(base.convert("RGB"))

#贴回(x1p,y1p,x2p,y2p)
target_w, target_h = (x2p - x1p), (y2p - y1p)
out_roi_resized = out_roi.resize((target_w, target_h), Image.LANCZOS)
out_np = np.array(out_roi_resized.convert("RGB"))
#左上+右下同时对齐
assert out_np.shape[1] == target_w and out_np.shape[0] == target_h
base_np[y1p:y2p, x1p:x2p, :] = out_np


# 保存
os.makedirs("out", exist_ok=True)
composited = Image.fromarray(base_np, mode="RGB")
composited.save("out/out.png")
roi.save("out/roi.png") # 裁下来的原始ROI
out_roi.save("out/roi_ai.png") # AI生成的ROI
print("Saved")

  0%|          | 0/50 [00:00<?, ?it/s]

/tmp/ipython-input-2812121065.py:45: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  composited = Image.fromarray(base_np, mode="RGB")


Saved


In [ ]:
!pip -q install diffusers>=0.35.1 transformers accelerate safetensors pillow

import os, torch
from PIL import Image, ImageDraw
from diffusers import QwenImageEditPipeline

print(torch.__version__)

2.8.0+cu126


In [ ]:
# 用工业背景生成灭火器
from pathlib import Path
import random
# YOLO
def yolo_line(x1, y1, x2, y2, W, H, cls_id):
    cx = ((x1 + x2) / 2) / W
    cy = ((y1 + y2) / 2) / H
    bw = (x2 - x1) / W
    bh = (y2 - y1) / H
    return f"{cls_id} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n"

# generate
def generate_extinguishers(
    base_path: str,
    roi_box: tuple,
    n_total: int = 50,
    num_images_per_prompt: int = 5,
    seed: int = 42
):

    # 输出目录
    root = Path("dataset")
    img_dir = root / "images" / "train"
    lbl_dir = root / "labels" / "train"
    img_dir.mkdir(parents=True, exist_ok=True)
    lbl_dir.mkdir(parents=True, exist_ok=True)

    # 类别：灭火器（class_id=0）
    CLASS_ID = 0

    # 背景与ROI
    base = Image.open(base_path).convert("RGB")
    W, H = base.size
    x1, y1, x2, y2 = map(int, roi_box)
    roi = base.crop((x1, y1, x2, y2))
    target_w, target_h = (x2 - x1), (y2 - y1)

    # Prompt
    prompt = (
        "Insert a handheld red fire extinguisher,"
        "black hose and metal nozzle, visible pressure gauge, "
        "match the corridor lighting and perspective, "
        "soft shadow on the floor and subtle reflection on the wall tiles, "
        "photorealistic and seamlessly integrated"
    )
    negative_prompt = (
        "cartoon, painting, illustration, logo, watermark, text overlay, "
        "deformed, unrealistic texture, mismatched lighting, shadow mismatch"
    )

    TRUE_CFG = 4.0
    STEPS = 50

    total = 0
    call_idx = 0
    rng = random.Random(seed)

    while total < n_total:
        batch_size = min(num_images_per_prompt, n_total - total)

        # 换一个随机种子以提升多样性
        this_seed = rng.randint(0, 2**31 - 1)
        gen = torch.Generator(device="cuda").manual_seed(this_seed)

        out_imgs = edit(
            image=roi,
            prompt=prompt,
            negative_prompt=negative_prompt,
            true_cfg_scale=TRUE_CFG,
            num_inference_steps=STEPS,
            generator=gen,
            num_images_per_prompt=batch_size
        ).images

        for j, out_roi in enumerate(out_imgs):
            out_roi = out_roi.resize((target_w, target_h), Image.LANCZOS)
            # 贴回
            composited = base.copy()
            composited.paste(out_roi, (x1, y1))

            # 文件名
            stem = f"{Path(base_path).stem}_extinguisher_{call_idx}_{j}"
            img_p = img_dir / f"{stem}.jpg"
            lbl_p = lbl_dir / f"{stem}.txt"

            # 保存图像
            composited.save(img_p, quality=95)

            # 保存YOLO标签
            with open(lbl_p, "w") as f:
                f.write(yolo_line(x1, y1, x2, y2, W, H, CLASS_ID))

            total += 1

        call_idx += 1

    print(f" 生成完成：{total} 张")
    print(f" 图片目录：{img_dir}")
    print(f" 标签目录：{lbl_dir}")

In [ ]:
base_path = "back.png"                 # 背景图
x1, y1, x2, y2 = 846, 552, 967, 971    # 你的 ROI 坐标（按需改）
generate_extinguishers(
    base_path=base_path,
    roi_box=(x1, y1, x2, y2),
    n_total=50,                 # 生成 50 张
    num_images_per_prompt=5,    # 每批 5 张
    seed=123
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

 生成完成：50 张
 图片目录：dataset/images/train
 标签目录：dataset/labels/train


In [ ]:
import os
import shutil
import random
from pathlib import Path

source_images_dir = "/content/data/images/train"
source_labels_dir = "/content/data/labels/train"
output_dir = "/content/data"

if not os.path.exists(source_images_dir):
    print(f"错误：图片目录不存在 - {source_images_dir}")
    exit()

if not os.path.exists(source_labels_dir):
    print(f"错误：标签目录不存在 - {source_labels_dir}")
    exit()

image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.gif', '*.tiff']
image_files = []

for ext in image_extensions:
    image_files.extend(Path(source_images_dir).glob(ext))
    image_files.extend(Path(source_images_dir).glob(ext.upper()))

label_files = list(Path(source_labels_dir).glob("*.txt"))

image_label_pairs = []

for image_file in image_files:
    image_name = image_file.stem
    label_file = Path(source_labels_dir) / f"{image_name}.txt"

    if label_file.exists():
        image_label_pairs.append((image_file, label_file))

if len(image_label_pairs) == 0:
    print("没有找到匹配的图片和标签文件")
    exit()

random.seed(42)
random.shuffle(image_label_pairs)

total_count = len(image_label_pairs)
train_count = int(total_count * 0.8)
test_count = int(total_count * 0.1)
val_count = total_count - train_count - test_count

train_pairs = image_label_pairs[:train_count]
test_pairs = image_label_pairs[train_count:train_count + test_count]
val_pairs = image_label_pairs[train_count + test_count:]

# 创建COCO8格式的目录结构
output_images_dir = Path(output_dir) / "images"
output_labels_dir = Path(output_dir) / "labels"

# 创建test目录
test_images_dir = output_images_dir / "test"
test_labels_dir = output_labels_dir / "test"
test_images_dir.mkdir(parents=True, exist_ok=True)
test_labels_dir.mkdir(parents=True, exist_ok=True)

for image_file, label_file in test_pairs:
    shutil.copy2(image_file, test_images_dir / image_file.name)
    shutil.copy2(label_file, test_labels_dir / label_file.name)

# 创建val目录
val_images_dir = output_images_dir / "val"
val_labels_dir = output_labels_dir / "val"
val_images_dir.mkdir(parents=True, exist_ok=True)
val_labels_dir.mkdir(parents=True, exist_ok=True)

for image_file, label_file in val_pairs:
    shutil.copy2(image_file, val_images_dir / image_file.name)
    shutil.copy2(label_file, val_labels_dir / label_file.name)

print("数据集分割完成！")

数据集分割完成！


In [ ]:
import yaml
import os

dataset_path = "/content/data"
classes_file = "/content/data/labels/classes.txt"
output_yaml = "/content/data.yaml"

if not os.path.exists(classes_file):
    print(f"错误：classes.txt文件不存在 - {classes_file}")
    print("请确保classes.txt文件存在并包含类别名称")
    exit()

classes = []
with open(classes_file, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip()) == 0:
            continue
        classes.append(line.strip())

number_of_classes = len(classes)

config = {
    'path': dataset_path,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': number_of_classes,
    'names': classes
}

with open(output_yaml, 'w', encoding='utf-8') as f:
    yaml.dump(config, f, sort_keys=False)

print(f"YAML配置文件已创建: {output_yaml}")

print('\n文件内容:\n')
print(open(output_yaml).read())

YAML配置文件已创建: /content/data.yaml

文件内容:

path: /content/data
train: images/train
val: images/val
test: images/test
nc: 1
names:
- '0: fire extinguishers'



In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.0 MB/s eta 0:00:00


In [ ]:
!yolo detect train data=/content/data.yaml model=yolov8n.pt epochs=50 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.208 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, kera